In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchsummary import summary
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from os import walk
from skimage import io
from skimage.transform import resize as skresize

In [2]:
class class_net(torch.nn.Module):
    def __init__(self,n_class = 10, frozen = 0):
        super(class_net, self).__init__()
        self.net_vgg = models.vgg16(pretrained=True)
        
        ct = 0
        
        for i in range(0,frozen):
            if (hasattr(self.net_vgg.features[i], 'weight')):
                self.net_vgg.features[i].weight.requires_grad = False
        
        self.net_vgg.classifier = nn.Sequential(
                                    nn.Linear(25088,4096),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(p=0.5,inplace = False),
                                    nn.Linear(4096,1024),
                                    nn.ReLU(inplace=True),
                                    nn.Dropout(p=0.5,inplace = False),
                                    nn.Linear(1024,n_class),
                                    nn.LogSoftmax(dim=1))
        
        
        
        
        

    
    def forward(self, x):
        return self.net_vgg(x)
        

In [4]:
class Resize(object):


    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, image):
        
        image = skresize(image, self.output_size, anti_aliasing=False)

        return image
    
class RandomCrop(object):


    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, image):
        
        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w,:]

        return image

class RandomFlip(object):
    def __init__(self, prob):
        self.prob = prob
        
    def __call__(self, image):
        coin = np.random.uniform()
        if(coin < self.prob):
            image = image[::-1,:,:]
        
        coin = np.random.uniform()
        if(coin < self.prob):
            image = image[:,::-1,:]

        return image

    
class ToTensor(object):
        
    def __call__(self, image):
        ss = np.shape(image)
        image_new =  np.zeros((ss[2],ss[0],ss[1]))
        
        image_new[0,:,:] = image[:,:,0]
        image_new[1,:,:] = image[:,:,1]
        image_new[2,:,:] = image[:,:,2]
        
        image_new = torch.from_numpy(image_new)
        
        return image_new

In [5]:
class Clean_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, transform=None):

        f = []
        for (dirpath, dirnames, filenames) in walk(root_dir):
            f.extend(dirnames)
            break
            
        self.transform = transform
        
        self.class_names = f
        self.labels = []
        self.images = []
        
        for i in range(0,len(self.class_names)):
            f = []
            for (dirpath, dirnames, filenames) in walk(root_dir + self.class_names[i]):
                f.extend(filenames)
                break
            for j in range(0,len(f)):
                self.images.append(root_dir + self.class_names[i] + '/' + f[j])
                self.labels.append(int(self.class_names[i]))
            

    def __len__(self):
        return len(self.images)
    
        

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = io.imread(self.images[idx])
        
        label = self.labels[idx]

        if(self.transform):
            image = self.transform(image)

        return image, label
        
        


In [8]:
data_trans = transforms.Compose([Resize((260,260)),
                                RandomCrop((224,224)),
                               RandomFlip(0.3),
                                ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

train_dataset = Clean_dataset('new_hard/train/', data_trans)
traindataloader = DataLoader(train_dataset, batch_size=32,
                        shuffle=True, num_workers=3)

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    raise Exception("You requested GPU support, but there's no GPU on this machine")

    
#net = models.vgg16(pretrained=True)
#net = net.to(device)
#net = nn.Sequential(*list(net.children())[:-1])

net = class_net(frozen = 31)
net = net.to(device)
#summary(net,(3,224,224))

In [ ]:
num_epochs = 200
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.MSELoss()